# 🚀 Ollama-SemanticKernel Automation Test Generation 🤖

Welcome to this exciting journey of AI-powered test automation! 🎉 This notebook demonstrates how to leverage the power of Ollama and Semantic Kernel to automatically generate C# Playwright page object models. Let's revolutionize the way we create automation tests! 💪

## 🌟 What We'll Accomplish

1. 🔧 Set up our environment with necessary packages
2. 🧠 Configure Ollama and Semantic Kernel
3. 📝 Load and process prompt templates
4. 🎭 Generate C# Playwright page object models from natural language descriptions
5. 🔍 Explore the generated code and understand its structure

## 🛠️ Technologies Used

- 🦙 Ollama: For running large language models locally
- 🧠 Semantic Kernel: Microsoft's framework for AI orchestration
- 🎭 Playwright: For web testing and automation
- 🔷 C#: Our primary programming language

## 💡 Key Concepts

- Few-shot learning: Using examples to guide AI output
- Prompt engineering: Crafting effective instructions for AI models
- Page Object Model: A design pattern for creating maintainable automation tests

Ready to dive in? Let's start coding and see AI in action! 🚀

# 📦 NuGet Package Installation

This cell sets up our development environment by installing the necessary NuGet packages for our project.

- **Microsoft.SemanticKernel**: This package provides the core functionality for building AI-powered applications. It allows us to integrate large language models (LLMs) into our C# code seamlessly.

- **OllamaSharp**: This package is a C# client for Ollama, enabling us to interact with Ollama's API and leverage its capabilities in our project.

By running this cell, we ensure that our notebook has access to all the required dependencies for generating C# Playwright page object models using AI.

In [14]:
#r "nuget:Microsoft.SemanticKernel"
#r "nuget:OllamaSharp"

Installed Packages Microsoft.SemanticKernel, 1.17.1 OllamaSharp, 2.1.1

# 📚 Importing Necessary Namespaces

This cell imports all the required namespaces for our project. Let's break down why each is important:

## 🔧 System Namespaces
- `System`: Provides fundamental classes and base classes for C#
- `System.IO`: Enables reading and writing to files and data streams
- `System.Net.Http`: Allows making HTTP requests, crucial for API interactions
- `System.Threading`: Provides classes for multi-threaded programming

## 🦙 Ollama Integration
- `OllamaSharp`: The main namespace for interacting with Ollama
- `OllamaSharp.Models.Chat`: Contains models specific to chat functionalities in Ollama

## 🧠 Semantic Kernel Setup
- `Microsoft.Extensions.DependencyInjection`: For setting up dependency injection
- `Microsoft.SemanticKernel`: The core namespace for Semantic Kernel functionality
- `Microsoft.SemanticKernel.ChatCompletion`: Specific to chat completion features in Semantic Kernel

By importing these namespaces, we're equipping our notebook with all the tools needed to interact with Ollama, utilize Semantic Kernel, and handle various system-level operations. This sets the stage for our AI-powered test generation adventure! 🚀

In [15]:
using System;
using System.IO;
using System.Net.Http;
using System.Threading;

using OllamaSharp;
using OllamaSharp.Models.Chat;

using Microsoft.Extensions.DependencyInjection;

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;

# 🔧 Configuration Settings

This cell defines crucial configuration settings for our AI-powered test generation project. Let's break down each variable:

## 🎨 `_outputFormat`
- Set to: `"Handlebars"`
- Purpose: Specifies the template engine used for formatting the AI's output.
- 💡 Handlebars is a popular templating language that allows for dynamic content insertion.

## 📁 `_promptsDirectory`
- Path: `"./resources/prompts/example-input"`
- Purpose: Points to the directory containing example inputs for few-shot learning.
- 🚀 These examples help guide the AI in generating accurate and relevant code.

## 📘 `_outputInstructDirectory`
- Path: `"./resources/prompts/output-instruct"`
- Purpose: Specifies the location of output instruction templates.
- 🎭 These instructions help format the AI's responses consistently.

## 📝 `_mainPromptFile`
- Path: `"./resources/prompts/llm.md"`
- Purpose: Indicates the file containing the main system prompt for the AI.
- 🧠 This prompt sets the context and provides general instructions for the AI.

These settings form the backbone of our project's configuration, allowing for easy customization and maintenance. By adjusting these paths, we can quickly adapt our system to use different prompts, examples, or output formats. 🛠️

In [16]:
static string _outputFormat = "Handlebars";
static string _promptsDirectory = "./resources/prompts/example-input";
static string _outputInstructDirectory = "./resources/prompts/output-instruct";
static string _mainPromptFile = "./resources/prompts/llm.md";

# 🏗️ Implementing Chat Completion Services

This cell defines two crucial classes for our AI-powered test generation project: `BaseOllamaChatCompletionService` and `GenericChatCompletionService`. Let's break down their structure and purpose:

## 🧱 BaseOllamaChatCompletionService

This abstract class serves as the foundation for our Ollama-based chat completion services.

Key Components:
- 🌐 `_httpClient`: For making HTTP requests to the Ollama API
- 📛 `_modelName`: Stores the name of the language model being used
- 📚 `Attributes`: A dictionary for storing additional attributes

Main Method:
- 🔄 `GetChatMessageContentsAsync`: 
  - Handles the core chat completion logic
  - Manages chat history and system prompts
  - Interacts with the Ollama API to generate responses

Notable Features:
- 🔒 Abstract method `GetSystemPromptAsync`: Allows subclasses to define their own system prompts
- 🚫 `GetStreamingChatMessageContentsAsync`: Placeholder for potential future streaming implementation

## 🎛️ GenericChatCompletionService

This concrete class extends `BaseOllamaChatCompletionService`, providing a simple implementation.

Key Features:
- 🏗️ Constructor: Initializes the service with a model URL and name
- 🗨️ `GetSystemPromptAsync`: Returns a basic system prompt

In [17]:
public abstract class BaseOllamaChatCompletionService : IChatCompletionService
{
    protected readonly HttpClient _httpClient;
    protected readonly string _modelName;
    
    public IReadOnlyDictionary<string, object?> Attributes { get; set; } = new Dictionary<string, object?>();

    /// <summary>
    /// Initializes a new instance of the <see cref="BaseOllamaChatCompletionService"/> class.
    /// </summary>
    /// <param name="modelUrl">The URL of the model.</param>
    /// <param name="modelName">The name of the model.</param>
    protected BaseOllamaChatCompletionService(string modelUrl, string modelName)
    {
        _modelName = modelName;
        _httpClient = new HttpClient
        {
            BaseAddress = new Uri(modelUrl),
            Timeout = TimeSpan.FromMinutes(2)
        };
    }


    /// <summary>
    /// Gets the chat message contents asynchronously.
    /// </summary>
    /// <param name="chatHistory">The chat history.</param>
    /// <param name="executionSettings">The execution settings.</param>
    /// <param name="kernel">The kernel.</param>
    /// <param name="cancellationToken">The cancellation token.</param>
    public virtual async Task<IReadOnlyList<ChatMessageContent>> GetChatMessageContentsAsync(
        ChatHistory chatHistory,
        PromptExecutionSettings? executionSettings = null,
        Kernel? kernel = null,
        CancellationToken cancellationToken = default)
    {
        var ollama = new OllamaApiClient(_httpClient, _modelName);
        var chat = new Chat(ollama, _ => { });

        // Add system message
        var systemPrompt = await GetSystemPromptAsync();
        await chat.SendAs(ChatRole.System, systemPrompt, cancellationToken);

        // Iterate through chatHistory Messages
        foreach (var message in chatHistory)
        {
            if (message.Role == AuthorRole.System)
            {
                await chat.SendAs(ChatRole.System, message.Content ?? string.Empty, cancellationToken);
            }
        }

        var lastMessage = chatHistory.LastOrDefault();
        string question = lastMessage?.Content ?? string.Empty;
        var history = (await chat.Send(question, cancellationToken)).ToArray();
        var chatResponse = history.Last().Content ?? string.Empty;

        chatHistory.AddAssistantMessage(chatResponse);

        return chatHistory;
    }

    /// <summary>
    /// Gets the streaming chat message contents asynchronously.
    /// </summary>
    /// <param name="chatHistory">The chat history.</param>
    /// <param name="executionSettings">The execution settings.</param>
    /// <param name="kernel">The kernel.</param>
    /// <param name="cancellationToken">The cancellation token.</param>
    public virtual IAsyncEnumerable<StreamingChatMessageContent> GetStreamingChatMessageContentsAsync(
        ChatHistory chatHistory,
        PromptExecutionSettings? executionSettings = null,
        Kernel? kernel = null,
        CancellationToken cancellationToken = default)
    {
        throw new NotImplementedException();
    }

    protected abstract Task<string> GetSystemPromptAsync();
}

public class GenericChatCompletionService : BaseOllamaChatCompletionService
{
    /// <summary>
    /// Initializes a new instance of the <see cref="GenericChatCompletionService"/> class.
    /// </summary>
    /// <param name="modelUrl">The URL of the model.</param>
    /// <param name="modelName">The name of the model.</param>
    public GenericChatCompletionService(string modelUrl, string modelName) : base(modelUrl, modelName)
    {

    }

    /// <summary>
    /// Gets the system prompt asynchronously.
    /// </summary>
    /// <returns>The system prompt.</returns>
    protected override Task<string> GetSystemPromptAsync()
    {
        return Task.FromResult("You are a helpful assistant.");
    }
}

# 🚀 PromptEngine: Powering Our AI's Understanding

The `PromptEngine` class is the heart of our (very simple) prompt management system. It handles loading, combining, and formatting the various components that guide our AI in generating accurate C# Playwright page object models. Let's break it down:

## 🧠 Key Methods

### 1. 📚 `LoadExamplesFromFilesAsync`
- 🎯 Purpose: Loads example prompts from markdown files
- 📂 Source: `_promptsDirectory`
- 🔢 Limit: Controlled by `maxExamples` parameter
- 🔍 Note: Helps in implementing few-shot learning

### 2. 📘 `LoadOutputInstructionsAsync`
- 🎯 Purpose: Loads instructions for formatting the AI's output
- 📂 Source: `_outputInstructDirectory`
- 🎨 Format: Determined by `_outputFormat` (e.g., Handlebars)

### 3. 📝 `LoadMainPromptAsync`
- 🎯 Purpose: Loads the main system prompt
- 📂 Source: `_mainPromptFile`
- ⚠️ Note: Throws an exception if the file is not found

### 4. 🔧 `GetSystemPromptWithExamplesAsync`
- 🎯 Purpose: Combines all components into a single, formatted prompt
- 🧩 Components: Main prompt, examples, output format, and instructions
- 🔢 Default: Loads up to 3 examples unless specified otherwise

The `PromptEngine` ensures that our AI receives well-structured, consistent instructions, enhancing the quality and reliability of the generated C# Playwright page object models. It's the bridge between our project's configuration and the AI's understanding of its task. 🌉🤖

In [18]:
/// <summary>
/// Simple prompt engine.
/// </summary>
public static class PromptEngine{
    /// <summary>
    /// Loads the examples from files asynchronously.
    /// </summary>
    /// <param name="maxExamples">The maximum number of examples to load.</param>
    /// <returns>The examples.</returns>
    public static async Task<string> LoadExamplesFromFilesAsync(int maxExamples)
    {
        var exampleFiles = Directory.GetFiles(_promptsDirectory, "*.md");
        var examples = new StringBuilder();

        var idx = 0;

        foreach (var file in exampleFiles)
        {
            if(idx >= maxExamples)
            {
                break;
            }

            examples.AppendLine($"Example: {Path.GetFileNameWithoutExtension(file)}");
            examples.AppendLine(await File.ReadAllTextAsync(file));
            examples.AppendLine();

            idx++;
        }

        return examples.ToString();
    }

    /// <summary>
    /// Loads the output instructions asynchronously.
    /// </summary>
    /// <returns>The output instructions.</returns> 
    public static async Task<string> LoadOutputInstructionsAsync()
    {
        var instructionFile = Path.Combine(_outputInstructDirectory, $"{_outputFormat}.md");
        if (File.Exists(instructionFile))
        {
            return await File.ReadAllTextAsync(instructionFile);
        }
        return string.Empty;
    }

    /// <summary>
    /// Loads the main prompt asynchronously.
    /// </summary>
    /// <returns>The main prompt.</returns>
    public static async Task<string> LoadMainPromptAsync()
    {
        if (File.Exists(_mainPromptFile))
        {
            return await File.ReadAllTextAsync(_mainPromptFile);
        }
        throw new FileNotFoundException($"Main prompt file not found: {_mainPromptFile}");
    }

    /// <summary>
    /// Gets the system prompt with examples asynchronously.
    /// </summary>
    /// <param name="maxExamples">The maximum number of examples to load.</param>
    /// <returns>The system prompt with examples.</returns>
    public static async Task<string> GetSystemPromptWithExamplesAsync(int maxExamples = 3)
    {
        var mainPrompt = await LoadMainPromptAsync();
        var examples = await LoadExamplesFromFilesAsync(maxExamples);
        var outputInstructions = await LoadOutputInstructionsAsync();

        return string.Format(mainPrompt, examples, _outputFormat, outputInstructions);
    }
}

# 🎭 Generating C# Playwright Page Object Models

This cell showcases the culmination of our project: using AI to generate C# Playwright page object models. Let's break down the process and examine the result.

## 🚀 The Generation Process

1. 📁 **Loading Prompts**
   - Reads all prompt files from `./resources/prompts/models`
   - Each file likely contains a description of a web page or component

2. 🧠 **Preparing the System Prompt**
   - Uses `PromptEngine.GetSystemPromptWithExamplesAsync()` to create a comprehensive prompt
   - Includes examples and instructions for the AI

3. 🤖 **Setting Up Ollama**
   - Creates a `GenericChatCompletionService` connected to a local Ollama instance
   - Uses the "llama3" model

4. 🔧 **Configuring Semantic Kernel**
   - Sets up a `Kernel` with the Ollama chat service

5. 🔁 **Processing Each Prompt**
   - Iterates through each loaded prompt
   - Creates a new `ChatHistory` for each prompt
   - Adds the system prompt and user input to the history
   - Generates a response using the chat service

6. 📤 **Output**
   - Prints the generated C# code to the console

## 🎨 The Generated Page Object Model

The AI has produced a `ShoppingCartPage` class, which is a comprehensive Playwright page object model. Key features include:

- 📦 **Base Structure**: Inherits from `BasePage` and uses Playwright's `IPage`
- 🧱 **Nested Classes**: `CartItemsList`, `CartItem`, and `CartItems` for structured element representation
- 🔍 **Locators**: Defined for various elements like product name, quantity input, and remove button
- 🛠️ **Methods**: 
  - `UpdateItemQuantityAsync`: For changing item quantities
  - `RemoveItemAsync`: For removing items from the cart
  - `ProceedToCheckoutAsync`: For moving to the checkout process

This cell demonstrates the power of AI in automating the creation of test infrastructure, potentially saving hours of manual coding and reducing errors in the process. 🎉

In [13]:
var prompts = Directory.EnumerateFiles("./resources/prompts/models")
    .Select(promptFile => File.ReadAllText(promptFile));

var sysPrompt = await PromptEngine.GetSystemPromptWithExamplesAsync();

// await SolveProblemWithToT(input);
var ollamaChat = new GenericChatCompletionService("http://localhost:11434", "llama3");

var builder = Kernel.CreateBuilder();

builder.Services.AddKeyedSingleton<IChatCompletionService>("ollamaChat", ollamaChat);
Kernel kernel = builder.Build();


foreach(var prompt in prompts){
    var chat = kernel.GetRequiredService<IChatCompletionService>();

    var history = new ChatHistory();
    history.AddSystemMessage(sysPrompt);
    history.AddUserMessage(prompt);

    var result = await chat.GetChatMessageContentsAsync(history);

    Console.WriteLine(result[^1].Content);
}

using Microsoft.Playwright;

public class ShoppingCartPage : BasePage
{
    private readonly IPage _page;

    public ShoppingCartPage(IPage page) : base(page)
    {
        _page = page;
    }

    public class CartItemsList : BaseElement
    {
        public CartItemsList(ILocator locator) : base(locator)
        {
        }

        public new async Task<IReadOnlyList<CartItems>> AllAsync()
        {
            var all = await base.AllAsync();
            return all.Select(a => new CartItems(a)).ToList();
        }
    }

    public class CartItem : BaseElement
    {
        private readonly ILocator _locator;

        public CartItem(ILocator locator) : base(locator)
        {
            _locator = locator;
        }

        public ILocator ProductName => _locator.Locator(".product-name");
        public ILocator QuantityInput => _locator.Locator(".quantity-input");
        public ILocator RemoveButton => _locator.Locator(".remove-item");
    }

    public class CartItems : Base